In [0]:
%pip install azure-keyvault pycryptodome -q

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("uc_service_credential", defaultValue="production-akv")
dbutils.widgets.text("metastore_id", defaultValue=sql("SELECT element_at(split(current_metastore(), ':'), -1) AS metastore").first().metastore)

service_credential = dbutils.widgets.get("uc_service_credential")
metastore_id = dbutils.widgets.get("metastore_id")

In [0]:
from azure.keyvault.keys import KeyClient, KeyType

credential = dbutils.credentials.getServiceCredentialsProvider(service_credential)
akv_url = "https://aweaver-west-europe.vault.azure.net/"
client = KeyClient(vault_url=akv_url, credential=credential)
keys = client.list_properties_of_keys()

key = client.create_key(name="aweaver-test-aes", key_type=KeyType.oct_hsm)

print(key.id)
print(key.name)
print(key.key_type)

#rsa_key = client.create_rsa_key("aweaver-test-cmk", size=2048)
# print(rsa_key.name)
# print(rsa_key.key_type)

In [0]:
key = client.create_oct_key("aweaver-aes", size=256, hardware_protected=True)

print(key.id)
print(key.name)
print(key.key_type)

In [0]:
from Crypto.Random import get_random_bytes  

key_bytes = get_random_bytes(32)
key_bytes

In [0]:
#aes_256= 'A256KW'

In [0]:
from azure.keyvault.keys.crypto import CryptographyClient
from azure.keyvault.keys.crypto import KeyWrapAlgorithm

crypto_client = CryptographyClient(key=client.get_key("aweaver-ec"), credential=credential)

# # the result holds the encrypted key and identifies the encryption key and algorithm used
result = crypto_client.wrap_key(KeyWrapAlgorithm.aes_256, key_bytes)
encrypted_key = result.encrypted_key
print(result.key_id)
print(result.algorithm)